# DRCT 후처리 코드

### 1. 필요 라이브러리 설치

In [2]:
!pip install einops

In [5]:
!pip install basicsr

  Preparing metadata (setup.py) ... done
  Using cached torch-2.4.1-cp38-cp38-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached triton-3.0.0-1-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 29.7 MB/s eta 0:00:00
Using cached torch-2.4.1-cp38-cp38-manylinux1_x86_64.whl (797.1 MB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
Using cached triton-3.0.0-1-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (209.4 MB)
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214818 sha256=40f6f2dbdf1e68c74b9b430120f493e53c5fae66aecd356afe858e30f12b7498
  Stored in directory: /home/kwy

### 2. 깃클론하기

In [2]:
!git clone https://github.com/ming053l/DRCT.git

'DRCT'에 복제합니다...
remote: Enumerating objects: 458, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 458 (delta 28), reused 0 (delta 0), pack-reused 406 (from 1)
오브젝트를 받는 중: 100% (458/458), 8.50 MiB | 10.78 MiB/s, 완료.
델타를 알아내는 중: 100% (255/255), 완료.


In [1]:
!pwd

/home/kwy00/song


### 3. DRCT로 이미지 해상도 높이기

In [1]:
# %run 매직 명령어 사용
!python ./DRCT/inference.py \
    --input /home/kwy00/song/lama-with-refiner/submission/fifth \
    --output /home/kwy00/song/drct-combine3 \
    --model_path /home/kwy00/song/DRCT-L_X4.pth


/home/kwy00/anaconda3/envs/song/lib/python3.8/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
./DRCT/inference.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializatio

### 4. 512X512 사이즈로 변경후 .Zip 폴더 생성

In [2]:
import os
import zipfile
from PIL import Image

def process_images(input_folder, output_zip_path):
    # 임시 폴더 생성
    temp_folder = os.path.join(input_folder, "processed_images")
    os.makedirs(temp_folder, exist_ok=True)
    
    # 이미지 처리
    for file_name in os.listdir(input_folder):
        input_file_path = os.path.join(input_folder, file_name)
        
        # 이미지 파일만 처리
        if not os.path.isfile(input_file_path) or not file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            continue
        
        # 이름 변경
        new_name = file_name.replace("_DRCT-L_X4", "")  # "_DRCT-L_X4" 제거
        output_file_path = os.path.join(temp_folder, new_name)
        
        # 이미지 크기 변경
        with Image.open(input_file_path) as img:
            new_size = (img.width // 4, img.height // 4)  # 크기를 4배 축소
            img_resized = img.resize(new_size, Image.LANCZOS)  # LANCZOS 사용
            img_resized.save(output_file_path)
    
    # 압축 파일 생성
    with zipfile.ZipFile(output_zip_path, 'w') as zipf:
        for file_name in os.listdir(temp_folder):
            file_path = os.path.join(temp_folder, file_name)
            zipf.write(file_path, arcname=file_name)  # 압축 파일에 저장
    
    # 임시 폴더 삭제
    for file_name in os.listdir(temp_folder):
        os.remove(os.path.join(temp_folder, file_name))
    os.rmdir(temp_folder)
    print(f"Processed images saved in: {output_zip_path}")

# 입력 및 출력 경로 설정
input_folder = "/home/kwy00/song/drct-combine3"  # 변경 전 폴더 경로
output_zip_path = "/home/kwy00/song/fifth_DRCT_L_6286.zip"  # 압축 파일 저장 경로

# 함수 실행
process_images(input_folder, output_zip_path)


Processed images saved in: /home/kwy00/song/fifth_DRCT_L_6286.zip
